In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa

import tensorflow.keras as keras
import matplotlib.pyplot as plt
import os
import io
import pickle

import sys
sys.path.append(os.path.join('..', '..'))

from global_params import *

In [2]:
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
train_path = os.path.join('..', '..', 'data', 'train')
test_path = os.path.join('..', '..', 'data', 'test')

In [4]:
# Hyperparameters
train_batch_size = 32
test_batch_size = 32

In [5]:
train_batches = keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255
).flow_from_directory(
    train_path,
    target_size=(28, 28),
    batch_size=train_batch_size,
    class_mode='categorical'
)

Found 3397 images belonging to 158 classes.


In [6]:
train_batches.class_indices

{'Abdullah_Gul': 0,
 'Adrien_Brody': 1,
 'Alejandro_Toledo': 2,
 'Alvaro_Uribe': 3,
 'Amelie_Mauresmo': 4,
 'Andre_Agassi': 5,
 'Andy_Roddick': 6,
 'Angelina_Jolie': 7,
 'Ann_Veneman': 8,
 'Anna_Kournikova': 9,
 'Ari_Fleischer': 10,
 'Ariel_Sharon': 11,
 'Arnold_Schwarzenegger': 12,
 'Atal_Bihari_Vajpayee': 13,
 'Bill_Clinton': 14,
 'Bill_Gates': 15,
 'Bill_McBride': 16,
 'Bill_Simon': 17,
 'Britney_Spears': 18,
 'Carlos_Menem': 19,
 'Carlos_Moya': 20,
 'Catherine_Zeta-Jones': 21,
 'Charles_Moose': 22,
 'Colin_Powell': 23,
 'Condoleezza_Rice': 24,
 'David_Beckham': 25,
 'David_Nalbandian': 26,
 'Dick_Cheney': 27,
 'Dominique_de_Villepin': 28,
 'Donald_Rumsfeld': 29,
 'Edmund_Stoiber': 30,
 'Eduardo_Duhalde': 31,
 'Fidel_Castro': 32,
 'George_HW_Bush': 33,
 'George_Robertson': 34,
 'George_W_Bush': 35,
 'Gerhard_Schroeder': 36,
 'Gloria_Macapagal_Arroyo': 37,
 'Gonzalo_Sanchez_de_Lozada': 38,
 'Gordon_Brown': 39,
 'Gray_Davis': 40,
 'Guillermo_Coria': 41,
 'Halle_Berry': 42,
 'Hamid_Kar

In [7]:
test_batches = keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255
).flow_from_directory(
    test_path,
    target_size=(28, 28),
    batch_size=test_batch_size,
    class_mode='categorical'
)

Found 927 images belonging to 158 classes.


In [8]:
# More hyperparameters
learning_rate = 1e-5
epochs = 100

# Math works out so that one epoch equals one pass through the dataset.
train_steps_per_epoch = train_batches.n // train_batch_size
test_steps_per_epoch = test_batches.n // test_batch_size

In [9]:
# Callback for early stopping
es_callback = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0001,
    patience=5,
    restore_best_weights=True
)

In [10]:
model = keras.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu', input_shape=(28, 28, 3)),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Dropout(0.3),
    keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Dropout(0.3),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation=None),
    keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
], name='zeiler_fergus')

In [11]:
model.summary()

Model: "zeiler_fergus"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4128      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1568)            

In [12]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.0001),
    loss=tfa.losses.TripletSemiHardLoss(),
    metrics=['accuracy']
)

In [13]:
history = model.fit(
    train_batches,
    epochs=5
)

Epoch 1/5


InvalidArgumentError:  Input to reshape is a tensor with 5056 values, but the requested shape has 32
	 [[{{node TripletSemiHardLoss/PartitionedCall/Reshape}}]] [Op:__inference_train_function_1364]

Function call stack:
train_function
